# Simultaneous Localization and Mapping (SLAM)

The SLAM (Simultaneous Localization and Mapping) problem is the problem of building a map of the environment while simultaneously determining the robot’s position relative to this map. This problem is also referred to as Concurrent Mapping and Localization (CML). A mobile robot moves through an unknown environment, starting at a known location (often given by coordinates {0,0,0}). Its motion is uncertain due to the robot’s model and sensor noise.
As it roams, the robot can detect the environment using a form of observation measurement. To compute the map and the robot’s location within this map, the probability of accuracy of the data must be included. 

SLAM has received considerable attention since the 1990's. Several approaches have been proposed such as Kalman and Particle filter SLAM. Gmapping is arguably the most popular software implementation of SLAM. It employs a Rao-Blackwellized particle filter (RBPF) and builds a 2D map based on laser scan data. The code and documentation are available on the [OpenSLAM website](http://openslam.org/gmapping.html). It is very easy to integrate into ROS, and can even be run on a low cost robot-platform like the Turtlebot 3. This section will demonstrate how to start SLAM on this robot, how to visualize the data and how to record it for future use.

## Visualization in rviz

rviz is a 3D visualization tool for ROS. It is included in a full desktop install of ROS, but can of course be installed separately as well. rviz can display robot positions (with a complete robot model if it is provided in a URDF file), laser scans, point clouds and much information that is being published by ROS nodes. To start rviz, a roscore needs to be running:

In [2]:
!roscore

... logging to /home/robin/.ros/log/88de9384-1306-11e8-9f67-34e6d736f40a/roslaunch-robin-ubuntu-14208.log
Checking log directory for disk usage. This may take awhile.
Press Ctrl-C to interrupt
Done checking log file disk usage. Usage is <1GB.
]2;roscore
started roslaunch server http://robin-ubuntu:37745/
ros_comm version 1.12.7


SUMMARY

PARAMETERS
 * /rosdistro: kinetic
 * /rosversion: 1.12.7

NODES

auto-starting new master
process[master]: started with pid [14219]
ROS_MASTER_URI=http://robin-ubuntu:11311/
]2;roscore http://robin-ubuntu:11311/
setting /run_id to 88de9384-1306-11e8-9f67-34e6d736f40a
process[rosout-1]: started with pid [14232]
started core service [/rosout]
^C
[rosout-1] killing on exit


In [ ]:
In a separate terminal, you can start rviz:

In [7]:
!rviz

[ INFO] [1518781815.535708188]: rviz version 1.12.10
[ INFO] [1518781815.535764928]: compiled against Qt version 5.5.1
[ INFO] [1518781815.535781656]: compiled against OGRE version 1.9.0 (Ghadamon)
[ INFO] [1518781815.763632526]: Stereo is NOT SUPPORTED
[ INFO] [1518781815.763742586]: OpenGl version: 3 (GLSL 1.3).


You can use the left mouse button to rotate the camera, the right mouse button to zoom and the middle mouse button 
to pan. On the left hand side of the window the display options are listed. The global options are listed at the top. Parameters like background color and frame rate can be adjusted here. There is also an option called 'Fixed Frame', which referes to the static coordinate frame relative to which all other information is referenced. By default, the frame 'odom' is selected, but 'base_footprint', 'world' and 'map' are also often used. The fixed frame should not be moving (e.g. it should not be attached to a mobile robot), doing so might result in erroneous visualization. 

Below the global options, you can adjust the settings of the visualization grid, such as the cell size and the color. Below that, you will find the only topic that is shown by default, namely the /tf topic. tf is an abbreviation for transfer frame, and contains information about the relative location of all coordinate frames in the system. Eventually, all frames should be directly or indirectly referenced relative to the fixed frame, thus building a so called 'tf-tree'.

At the bottom left of the rviz window, the 'Add' button allows you to visualize additional topics. By selecting the LaserScan message type for example, an additional display option will appear. You can then set parameters like the laserscan topic and the size and color of the markers. You can add additional display options in a similar manner. You can even have two display options of the same type (for example, 2 camera displays). This option only makes sense when the data being displayed originates from different topics though.

Once you have added the desired display options, you can save your configuration as a .rviz file. Doing so will allow you to have the exact same visualization window in the future (and thus prevents having to add the same topics over and over again). To save your rviz file:

1. Select the 'file' option in the window bar of the rviz panel. 
2. Select 'save config as'. 
3. Choose an appropriate name and location for your file. Select the default location (/home/ROS-workshop/.rviz) and 'example' as a filename to follow along with the rest of the chapter.

You can now safely close rviz. To open the same configuration file in rviz, you can either type the ```rviz``` command in a terminal, then navigate to the file be selecting 'file' and then 'open'. Alternatively, you can pass the configuration file as an argument when opening rviz:

In [9]:
!rviz rviz  -d $HOME/.rviz/example.rviz

[ INFO] [1518782317.672422492]: rviz version 1.12.10
[ INFO] [1518782317.672495573]: compiled against Qt version 5.5.1
[ INFO] [1518782317.672513195]: compiled against OGRE version 1.9.0 (Ghadamon)
[ INFO] [1518782317.896427611]: Stereo is NOT SUPPORTED
[ INFO] [1518782317.896597588]: OpenGl version: 3 (GLSL 1.3).


The argument after ```-d``` should be the path to the rviz file. If everything works correctly, you should see the display options you added earlier in stead of the default window.